In [8]:
import csv
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
import librosa
import librosa.display
import wave
import os
import matplotlib.pyplot as plt
import multiprocessing 

In [9]:
SOURCE_FOLDER= '/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/'

In [10]:
OUTPUT_FOLDER_CLASSES = './data/music/Artists/'

In [11]:
Labels = [f.name for f in os.scandir(SOURCE_FOLDER) if f.is_dir()]
SubFolders = [f.path for f in os.scandir(SOURCE_FOLDER) if f.is_dir()]
for sub in SubFolders:
    print(sub.split("/")[len(sub.split("/"))-1])

Weezer
The Zombies
Zigeunerweisen
Zeke
Zao
Youth Group
Yann Tiersen
Winter Solstice
The White Stripes
White Stripes
Wham!
Western Waste
The Weakerthans
Wayne Kramer
War Of Ages
Voodoo Glow Skulls
Various Artists
Nelly Furtado
KROQ
Van Halen
The Used
Unwritten Law
The Unseen
Unknown
1208
4 Strings
A Dozen Furies
Aerosmith
AFI
Age of Ruin
Agnelli & Nelson
Alexisonfire
Alice Cooper
Alicia Keys
All
All Shall Perish
All That Remains
Alove for Enemies
American Hi-Fi
Amr Diab - Dj Driss
Anastacia
A New Found Glory
Anthrax
Apples in Stereo
Aretha Franklin
Ario - ( www.Bia2.com )
Arome
artist
ASG
Ashlee Simpson
As I Lay Dying
At All Cost
Atmosphere
Audioslave
Audioslave Featuring Maynard
August Burns Red
Authority Zero
Avenged Sevenfold
Aventura
Ayla
Bach
Bad Boys Blue
Bad Religion
Bayside
Beastie Boys
Beatsteaks
Bebo & Cigala
Beck
Becoming The Archetype
Beethoven
Bert Kaempfert
Bethoven
big bad voodoo daddy
Big Tittie Nickie - D.O.C. - Dr. Dre - Samara - Snoop Dogg
Black My Heart
Black Sabbath

In [12]:
if not os.path.exists(OUTPUT_FOLDER_CLASSES):
    os.makedirs(OUTPUT_FOLDER_CLASSES)
PathDictrionary={}
for i in Labels:
    LabeldDirectory = OUTPUT_FOLDER_CLASSES+i+"/"
    PathDictrionary[i]=LabeldDirectory
    if not os.path.exists(LabeldDirectory):
        os.makedirs(LabeldDirectory)

In [14]:
### New Multithreaded
def log_mel_spec_tfm(fname, src_path, dst_path):
    print(src_path)
    x=[]
    sample_rate=0
    try:
        x ,sample_rate = librosa.load(src_path)
    except:
        print(fname+" Could not be computed!")
        return
    print(len(x),fname)
    n_fft = 1024
    hop_length = 256
    n_mels = 224
    fmin = 20
    fmax = sample_rate / 2 
    
    mel_spec_power = librosa.feature.melspectrogram(x, sr=sample_rate, n_fft=n_fft, 
                                                    hop_length=hop_length, 
                                                    n_mels=n_mels, power=2.0,
                                                   fmin=fmin,fmax=fmax)
    mel_spec_db = librosa.power_to_db(mel_spec_power, ref=np.max) 
    #print("Befor",np.shape(mel_spec_db))
    width = np.shape(mel_spec_db)[1]
    #print(width,int(np.ceil(width/n_mels)))
    if width >= n_mels:
        for i in range(0,int(np.ceil(width/n_mels))):
            out= np.zeros(shape=(224,224))
            if((i+1)*n_mels>width):
                out = mel_spec_db[:,width-n_mels:]
            else:
                out = mel_spec_db[:,(i)*n_mels:(i+1)*n_mels]
            plt.imsave(os.path.join(dst_path,(fname.replace(".wav",'-').replace(".m4a",'-').replace(".mp3",'-') +str(i)+ '.png')), out) 

#TEST_PATH_FILE="/Volumes/Macintosh HD/Users/dg/Music/iTunes/iTunes Media/Music/Celldweller/End of an Empire (Deluxe Edition)"
#filename="1-01 End of an Empire.m4a"
#log_mel_spec_tfm(filename,TEST_PATH_FILE,"./testDir/")
#return;

counter = 0
for path in SubFolders:
    for root, dirs, filename in os.walk(path):# we disregard any subsorting
        if counter%100==0:
            print("Processed thus far",counter)
        filesToProcess=[]
        for f in filename:
            if f.endswith(".wav") or f.endswith(".m4a") or f.endswith(".mp3"):
                filesToProcess.append(f)
                counter +=1
            else:
                print("skipped: "+f)
        with multiprocessing.Pool(6) as p:
            outputDir = PathDictrionary[path.split("/")[len(path.split("/"))-1]]
            p.starmap(log_mel_spec_tfm, [(f,root+'/'+f,outputDir) for f in filesToProcess])
        print("Finished The Pool")
print("Done generating Spectra!")

Processed thus far 0
Finished The Pool
Processed thus far 0
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Weezer/Blue Album/01 - My Name is Jonas.mp3
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Weezer/Blue Album/02 - No One Else.mp3
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Weezer/Blue Album/03 - The World Has Turned and Left Me Here.mp3
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Weezer/Blue Album/04 - Buddy Holly.mp3
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Weezer/Blue Album/06 - Surf Wax America.mp3
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Weezer/Blue Album/05 - Undone (The Sweater Song).mp3
3514752 04 - Buddy Holly.mp3
4067136 02 - No One Else.mp3
4116096 06 - Surf Wax America.mp3
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Weezer/Blue Album/07 - Say It Aint So.mp3
4512384 01 - My Name is Jonas.mp3
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Weezer/Blue Album/08 - In The Garage.mp3
/Volumes/OTHER_PROJECTS/Goedicke_Neural

2682336 04 - Beautiful Rosmarin (Kreisler).m4a
3130336 03 - Girl With The Flaxen Hair.m4a
3374048 02 - The Spring Of Xin Jiang (Yuezhong Ma & Zhonghan Li).m4a
3323360 06 - Tapateado (Alberi).m4a
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Zigeunerweisen/Chinese Violin/07 - Zapateado (Sarasate).m4a
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Zigeunerweisen/Chinese Violin/08 - Fishermen Songs (Ancient Music).m4a
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Zigeunerweisen/Chinese Violin/09 - Poionaise Brillant (Wieniawsky).m4a
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Zigeunerweisen/Chinese Violin/10 - Souvenir (Drdla).m4a
5668832 01 - On Wings Of Songs (Mendelssohn).m4a
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Zigeunerweisen/Chinese Violin/11 - Zigeunerweisen (Sarasate).m4a
4789728 07 - Zapateado (Sarasate).m4a
4894176 10 - Souvenir (Drdla).m4a
12082656 05 - Introduction And Rondo Capriccioiso (Saint-Saen).m4a
7518688 08 - Fishermen Songs (Ancient M

/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/The White Stripes/Elephant/06 - I Want to Be the Boy to Warm Your Mother's Heart.mp3
3672000 04 - I Just Don't Know What to Do With Myself.mp3
3935232 05 - In the Cold, Cold Night.mp3
4055616 02 - Black Math.mp3
4422528 06 - I Want to Be the Boy to Warm Your Mother's Heart.mp3
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/The White Stripes/Elephant/07 - You've Got Her in Your Pocket.mp3
4932288 03 - There's No Home for You Here.mp3
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/The White Stripes/Elephant/08 - Ball and Biscuit.mp3
5112000 01 - Seven Nation Army.mp3
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/The White Stripes/Elephant/09 - The Hardest Button to Button.mp3
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/The White Stripes/Elephant/10 - Little Acorns.mp3
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/The White Stripes/Elephant/11 - Hypnotize.mp3
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/The 

/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Various Artists/Paul Oakenfold's Perfecto Chills Vol. 3/06 - Steppin' Out (Chill Out Mix).mp3
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Various Artists/Paul Oakenfold's Perfecto Chills Vol. 3/08 - Beauty 2.mp3
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Various Artists/Paul Oakenfold's Perfecto Chills Vol. 3/03 - Pushy.mp3
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Various Artists/Paul Oakenfold's Perfecto Chills Vol. 3/02 - Dream Machine (J Boogie Mix).mp3
5201742 01 - All Nigth Operator (Part 1).mp3
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Various Artists/Paul Oakenfold's Perfecto Chills Vol. 3/02 - Safe From Harm (Perfecto Mix).mp3
4638732 06 - Steppin' Out (Chill Out Mix).mp3
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Various Artists/Paul Oakenfold's Perfecto Chills Vol. 3/11 - All Over You (The Crazy Mix).mp3
5815320 08 - Beauty 2.mp3
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Vari

/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Unknown/Flowers in your hair/08 - All the leaves are brown.m4a
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Unknown/Flowers in your hair/07 - Sugar Pie.m4a
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Unknown/Flowers in your hair/04 - I Don't know why i love u but..i do.m4a
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Unknown/Flowers in your hair/06 - Nanana.m4a
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Unknown/Flowers in your hair/05 - How many roads must a man walk down?.m4a
3098080 04 - I Don't know why i love u but..i do.m4a
3594720 07 - Sugar Pie.m4a
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Unknown/Flowers in your hair/03 - On the Road Again.m4a
3210208 06 - Nanana.m4a
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Unknown/Flowers in your hair/02 - Raindrops keep fallin on my head.m4a
3472864 05 - How many roads must a man walk down?.m4a
3533792 08 - All the leaves are brown.m4a
4240352 09 

...

/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Less Than Jake/Anthem/11 - That's Why They Call It a Union.m4a
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Less Than Jake/Anthem/09 - Best Wishes to Your Black Lung.m4a
3034112 12 - Plastic Cup Politics.m4a
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Less Than Jake/Anthem/08 - Escape from the A-Bomb House.m4a
3854336 09 - Best Wishes to Your Black Lung.m4a
4056064 11 - That's Why They Call It a Union.m4a
4291584 10 - She's Gonna Break Soon.m4a
4885504 14 - Surrender [*].m4a
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Less Than Jake/Anthem/06 - Motown Never Sounded So Good.m4a
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Less Than Jake/Anthem/07 - The Upwards War and the Down Turned Cycle.m4a
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Less Than Jake/Anthem/05 - Short Fuse Burning.m4a
6484992 13 - The Brightest Bulb Has Burned Out-Screws Fall Out.m4a
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Le

/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Mannheim Steamroller/Fresh Aire/03 - Interlude I.m4a
3389920 05 - Interlude II.m4a
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Mannheim Steamroller/Fresh Aire/01 - Prelude.m4a
3361248 04 - Sonata.m4a
7300576 07 - Fresh Aire.m4a
2056672 01 - Prelude.m4a
3854816 02 - Chocolate Fudge.m4a
3867104 03 - Interlude I.m4a
Finished The Pool
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Mannheim Steamroller/Fresh Aire III/09 - Midnight on a Full Moon.m4a
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Mannheim Steamroller/Fresh Aire III/08 - The Sky.m4a
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Mannheim Steamroller/Fresh Aire III/07 - The Cricket.m4a
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Mannheim Steamroller/Fresh Aire III/06 - Interlude VI.m4a
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Mannheim Steamroller/Fresh Aire III/02 - Small Wooden Bach'ses.m4a
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC

Finished The Pool
Finished The Pool
Finished The Pool
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Metallica/And Justice For All/08 - To Live Is To Die.mp3
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Metallica/And Justice For All/09 - Dyers Eve.mp3
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Metallica/And Justice For All/06 - Harvester Of Sorrow.mp3
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Metallica/And Justice For All/05 - The Shortest Straw.mp3
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Metallica/And Justice For All/07 - The Frayed Ends Of Sanity.mp3
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Metallica/And Justice For All/04 - One.mp3
6910272 09 - Dyers Eve.mp3
7624512 06 - Harvester Of Sorrow.mp3
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Metallica/And Justice For All/03 - Eye of the Beholder.mp3
9865152 04 - One.mp3
10232640 07 - The Frayed Ends Of Sanity.mp3
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Metallica/And Justi

/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Mortal Treason/A Call To The Martyrs/05 - Feed On The Weak.mp3
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Mortal Treason/A Call To The Martyrs/04 - A Call To The Martyrs.mp3
4177446 04 - A Call To The Martyrs.mp3
4261236 06 - Bride's Last Kiss.mp3
4711938 09 - Acoustic.mp3
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Mortal Treason/A Call To The Martyrs/03 - War Within.mp3
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Mortal Treason/A Call To The Martyrs/02 - Walk In The Woods.mp3
5694486 07 - Beneath The Shadows.mp3
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Mortal Treason/A Call To The Martyrs/01 - Khampa Nomads.mp3
6540912 05 - Feed On The Weak.mp3
8262282 08 - Todd.mp3
4964190 03 - War Within.mp3
7917714 02 - Walk In The Woods.mp3
7347942 01 - Khampa Nomads.mp3
Finished The Pool
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Mortal Treason/Sunrise Over A Sea Of Blood/10 - Death Is The Beginning.mp3
/V

/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Mozart/250 A Celebration/24 - Alleluia.mp3
6102852 32 - Sonata for Piano.mp3
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Mozart/250 A Celebration/31 - Quartet for Flute and Strings.mp3
3586506 24 - Alleluia.mp3
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Mozart/250 A Celebration/23 - Symphony No 41 Jupiter.mp3
11234622 30 - Concerto for Clarnet and Orchestra.mp3
4714584 31 - Quartet for Flute and Strings.mp3
12184830 35 - Concerto for Flute and Harp.mp3
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Mozart/250 A Celebration/21 - Sonata No 11.mp3
12655230 34 - Concerto No 20.mp3
13429626 28 - Concerto No 3.mp3
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Mozart/250 A Celebration/29 - Sonata No 13.mp3
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Mozart/250 A Celebration/33 - Soave Sia Il Vento.mp3
/Volumes/OTHER_PROJECTS/Goedicke_NeuralMusic/MUSIC/Mozart/250 A Celebration/36 - Concerto No 23.mp3
8045016 23 -

In [ ]:
##### OLD COde not parallel
def log_mel_spec_tfm_old(fname, src_path, dst_path):
    x ,sample_rate = librosa.load(os.path.join(src_path,fname))
    
    n_fft = 1024
    hop_length = 256
    n_mels = 224
    fmin = 20
    fmax = sample_rate / 2 
    
    mel_spec_power = librosa.feature.melspectrogram(x, sr=sample_rate, n_fft=n_fft, 
                                                    hop_length=hop_length, 
                                                    n_mels=n_mels, power=2.0,
                                                   fmin=fmin,fmax=fmax)
    mel_spec_db = librosa.power_to_db(mel_spec_power, ref=np.max) 
    #print("Befor",np.shape(mel_spec_db))
    width = np.shape(mel_spec_db)[1]
    #print(width,int(np.ceil(width/n_mels)))
    if width >= n_mels:
        for i in range(0,int(np.ceil(width/n_mels))):
            out= np.zeros(shape=(224,224))
            if((i+1)*n_mels>width):
                out = mel_spec_db[:,width-n_mels:]
            else:
                out = mel_spec_db[:,(i)*n_mels:(i+1)*n_mels]
            plt.imsave(os.path.join(dst_path,(fname.replace(".wav",'-').replace(".m4a",'-').replace(".mp3",'-') +str(i)+ '.png')), out) 
#TEST_PATH_FILE="/Volumes/Macintosh HD/Users/dg/Music/iTunes/iTunes Media/Music/Celldweller/End of an Empire (Deluxe Edition)"
#filename="1-01 End of an Empire.m4a"
#log_mel_spec_tfm(filename,TEST_PATH_FILE,"./testDir/")
#return;

counter = 0
for path in SubFolders:
    for root, dirs, filename in os.walk(path):# we disregard any subsorting
        if counter>10:
            break;
        for f in filename:
            if f.endswith(".wav") or f.endswith(".m4a") or f.endswith(".mp3"):
                print("With catagory "+path.split("/")[len(path.split("/"))-1]+". Going to make a spectrum of "+f)
                log_mel_spec_tfm_old(f,root,PathDictrionary[path.split("/")[len(path.split("/"))-1]])
                counter +=1
            else:
                print("skipped"+f)
print("Done generating Spectra!")